In [2]:
pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 74.7 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 532.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.8 MB/s eta 0:00:00


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'Q\d+\.[A-Za-z]\.Sub')
    question_pattern = re.compile(r'^\d+\.\s')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    subheader_index = 1
    question_no = question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info + [marks])
        elif current_subheader:
            data.append([current_subheader, "Sub Question", "Subjective", ""] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0]
            subheader_index = 1
            question_no = ""
            question = ""
            marks = ""
            data.append([current_subheader, "Sub Question", "Subjective", ""] + [""] * (len(columns) - 5))
        elif question_pattern.match(line):
            add_question_entry()
            question_no = f"{current_subheader}.{subheader_index}"
            subheader_index += 1
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 4] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,2,2,15,0,,
1,.1,Actual Question,Subjective,This question paper contains 3 pages.,,,,,,,,,,,,,,,,
2,.2,Actual Question,Subjective,Answer to each new question to be started on a...,,,,,,,,,,,,,,,,
3,.3,Actual Question,Subjective,Figure in right hand side indicates full marks.,,,,,,,,,,,,,,,,
4,.4,Actual Question,Subjective,All Questions are compulsory.,,,,,,,,,,,,,,,,8
5,.5,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
6,.6,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
7,.7,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
8,.8,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
9,.9,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,


In [16]:
pip install ace_tools


ERROR: Could not find a version that satisfies the requirement ace_tools (from versions: none)
ERROR: No matching distribution found for ace_tools


In [10]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+ \([A-Z]\)')
    question_pattern = re.compile(r'^\d+\.\s')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    subheader_index = 1
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if current_subheader:
            if question_no:
                data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info[:6] + [marks] + additional_info[6:])
            else:
                data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            subheader_index = 1
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{subheader_index}"
            subheader_index += 1
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [29]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+ \([A-Z]\)')
    question_pattern = re.compile(r'^\d+\.\s')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    subheader_index = 1
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if current_subheader:
            if question_no:
                data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info[:6] + [marks] + additional_info[6:])
            else:
                data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            subheader_index = 1
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{subheader_index}"
            subheader_index += 1
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [28]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_path = "qb - sample.pdf"
text = extract_text_from_pdf(pdf_path)
print(text)


SVKM
SVKM’s UPG College of Arts, Sci. & Com.
Programme: BACHELOR OF MANAGEMENT STUDIES
Year: II/Additional Exam-even(Exam Year: 2023-2024)
Subject: Integrated Marketing Communication
Date: 18 Jun 2024 Time: 10:00 am to 12:30 pm (02:30 Hrs.)
Max. Marks: 75
FINAL EXAMINATION (Acad. Year:2023-2024)
Instructions:
1. This question paper contains 3 pages.
2. Answer to each new question to be started on a fresh page.
3. Figure in right hand side indicates full marks.
4. All Questions are compulsory.
Q.1 (A) Select the correct option from the multiple choices given: (Any Eight) (8)
1. What is IMC?
a) Inter-related Management Coordination
b) Integrated Marketing Communications
c) Integrated Members Chamber
d) International Marketing Contributors
2. Which of the following is an example of a digital marketing tool?
a) Hot air balloon
b) Graffiti
c) Newspaper Ads
d) Instagram Marketing
3. Which of the following is an example of a sales promotion tool?
a) Advertising
b) Public Relations
c) Personal

In [37]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = []

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-z]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if current_subheader:
            if question_no:
                data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info[:6] + [marks] + additional_info[6:])
            else:
                data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
1,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
2,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
3,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
4,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
5,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
6,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
7,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
8,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,
9,Q1.9,Actual Question,Subjective,Which of the following is not an objective of ...,To increase sales in the short term,To maintain a long lasting relationship with t...,To encourage repeat purchases,To introduce new products,,,,,,,,,,,,


In [39]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = []

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if current_subheader:
            if question_no:
                data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info[:6] + [marks] + additional_info[6:])
            else:
                data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text] + [""] * (len(columns) - 5))
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
1,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
2,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
3,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
4,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
5,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
6,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
7,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
8,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,
9,Q1.9,Actual Question,Subjective,Which of the following is not an objective of ...,To increase sales in the short term,To maintain a long lasting relationship with t...,To encourage repeat purchases,To introduce new products,,,,,,,,,,,,


In [42]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = []

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""

    def add_question_entry():
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question] + answers + additional_info[:6] + [marks] + additional_info[6:])
        else:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text] + [""] * (len(columns) - 5))
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
1,.1,Actual Question,Subjective,This question paper contains 3 pages.,,,,,,,,,,,,,,,,
2,.2,Actual Question,Subjective,Answer to each new question to be started on a...,,,,,,,,,,,,,,,,
3,.3,Actual Question,Subjective,Figure in right hand side indicates full marks.,,,,,,,,,,,,,,,,
4,.4,Actual Question,Subjective,All Questions are compulsory.,,,,,,,,,,,,,,,,
5,.4,Actual Question,Subjective,All Questions are compulsory.,,,,,,,,,,,,,,,,
6,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
7,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
8,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
9,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,


In [45]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = []

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-e]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""
    solve_placeholder_added = False

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, marks, solve_placeholder_added
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [marks] + additional_info[6:])
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    marks = marks_match.group(1)
                    sub_question_text = sub_question_text.replace(f"({marks})", "").strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text] + [""] * (len(columns) - 5))
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            question = question.replace(" OR", "")
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            question = question.replace(" OR", "")
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    df = pd.DataFrame(data, columns=columns)
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,.1,Actual Question,Subjective,This question paper contains 3 pages.,,,,,,,,,,,,,,,,
1,.2,Actual Question,Subjective,Answer to each new question to be started on a...,,,,,,,,,,,,,,,,
2,.3,Actual Question,Subjective,Figure in right hand side indicates full marks.,,,,,,,,,,,,,,,,
3,.4,Actual Question,Subjective,All Questions are compulsory.,,,,,,,,,,,,,,,,
4,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
5,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
6,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
7,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
8,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
9,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,


In [48]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = []

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""
    solve_placeholder_added = False

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, marks, solve_placeholder_added
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    marks = marks_match.group(1)
                    sub_question_text = sub_question_text.replace(f"({marks})", "").strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text] + [""] * (len(columns) - 5))
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            question = question.replace(" OR", "")
            marks_match = marks_pattern.search(question)
            if marks_match:
                marks = marks_match.group(1)
                question = question.replace(f"({marks})", "").strip()
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            question = question.replace(" OR", "")
            marks_match = marks_pattern.search(question)
            if marks_match:
                marks = marks_match.group(1)
                question = question.replace(f"({marks})", "").strip()
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        elif marks_match := marks_pattern.search(line):
            marks = marks_match.group(1)
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,.1,Actual Question,Subjective,This question paper contains 3 pages.,,,,,,,,,,,,,,,,
1,.2,Actual Question,Subjective,Answer to each new question to be started on a...,,,,,,,,,,,,,,,,
2,.3,Actual Question,Subjective,Figure in right hand side indicates full marks.,,,,,,,,,,,,,,,,
3,.4,Actual Question,Subjective,All Questions are compulsory.,,,,,,,,,,,,,,,,
4,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
5,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
6,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
7,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
8,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
9,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,


In [55]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    marks = ""
    solve_placeholder_added = False
    parsing_questions = False

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, marks, solve_placeholder_added
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            marks = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    marks = marks_match.group(1)
                    sub_question_text = sub_question_text.replace(f"({marks})", "").strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", "", marks, "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                marks = marks_match.group(1)
                question = question.replace(f"({marks})", "").strip()
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                marks = marks_match.group(1)
                question = question.replace(f"({marks})", "").strip()
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    marks_match = marks_pattern.search(line)
                    if marks_match:
                        marks = marks_match.group(1)
                        line = line.replace(f"({marks})", "").strip()
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [57]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    solve_placeholder_added = False
    parsing_questions = False

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, solve_placeholder_added
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [""] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [10]:
pip install transformers


In [22]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')  # Updated regex to match numbers within parentheses
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    solve_placeholder_added = False
    parsing_questions = False
    current_marks = ""

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, solve_placeholder_added, current_marks
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [current_marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            current_marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    current_marks = marks_match.group(1)
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", current_marks, "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [23]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')  # Updated regex to match numbers within parentheses
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    solve_placeholder_added = False
    parsing_questions = False
    current_marks = ""

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, solve_placeholder_added, current_marks
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [current_marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            current_marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    current_marks = marks_match.group(1)
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", current_marks, "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    # Remove specific phrases from the 'Question' column
    df['Question'] = df['Question'].str.replace(r'\s*\(7\)\s*OR\s*', '', regex=True)
    df['Question'] = df['Question'].str.replace(r'\s*\(8\)\s*', '', regex=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()
df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digi...,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sale...,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using I...,It allows for a fragmented and inconsistent me...,It creates confusion among customers about the...,It helps to create a unified and consistent me...,It increases the cost of marketing communicati...,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trad...,A deep freezer installed at a distribution out...,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a publ...,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of I...,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponso...,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a...,,,,,,,,,,,,


In [28]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-c]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')  # Updated regex to match numbers within parentheses
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    solve_placeholder_added = False
    parsing_questions = False
    current_marks = ""

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, solve_placeholder_added, current_marks
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [current_marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            current_marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([a-c]\).*?(?=\([a-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    current_marks = marks_match.group(1)
                data.append([f"{current_subheader}.{sub_question_no}", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", current_marks, "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    # Remove specific phrases from the 'Question' column
    df['Question'] = df['Question'].str.replace(r'\s*\(7\)\s*OR\s*', '', regex=True)
    df['Question'] = df['Question'].str.replace(r'\s*\(8\)\s*', '', regex=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()

# Set display options for wider output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df


,Question No,Question Type,Objective / Subjective,Question,Answer1,Answer2,Answer3,Answer4,Answer5,CorrectOption,Difficulty Level,Audio File Path,Image Name,Video File Path,OptionalSubQuestion,OutOfSubQuestion,Marks,ISOR,IsQuestionWiesRoundOff,Course Outcomes
0,Q1 Main,Main Question,Subjective,Compulsory,,,,,,,,,,,,2,2,15,0,
1,Q1,Sub Question,Subjective,Solve,,,,,,,,,,,,,,,,
2,Q1.1,Actual Question,Subjective,What is IMC?,Inter-related Management Coordination,Integrated Marketing Communications,Integrated Members Chamber,International Marketing Contributors,,,,,,,,,,,,
3,Q1.2,Actual Question,Subjective,Which of the following is an example of a digital marketing tool?,Hot air balloon,Graffiti,Newspaper Ads,Instagram Marketing,,,,,,,,,,,,
4,Q1.3,Actual Question,Subjective,Which of the following is an example of a sales promotion tool?,Advertising,Public Relations,Personal Selling,Coupons,,,,,,,,,,,,
5,Q1.4,Actual Question,Subjective,Which of the following is a benefit of using Integrated Marketing Communication?,It allows for a fragmented and inconsistent message to be delivered to customers.,It creates confusion among customers about the brand and its message.,It helps to create a unified and consistent message across all marketing channels.,It increases the cost of marketing communication efforts.,,,,,,,,,,,,
6,Q1.5,Actual Question,Subjective,Which of the following is an example of a trade promotion?,A deep freezer installed at a distribution outlet by the marketer,A buy-one-get-one-free offer,A free sample for testing,A customer loyalty program,,,,,,,,,,,,
7,Q1.6,Actual Question,Subjective,Which of the following is an example of a public relations activity?,A product giveaway,A celebrity endorsement,A press conference,A direct mail campaign,,,,,,,,,,,,
8,Q1.7,Actual Question,Subjective,Which of the following is not a component of Integrated Marketing Communication (IMC)?,Advertising,Product Lifecycle management,Public Relations,Personal Selling,,,,,,,,,,,,
9,Q1.8,Actual Question,Subjective,Which of the following is an example of sponsorship and events? ********** 1 **********,A billboard advertisement,A flyer designed for distribution,A charity walkathon organised by a sports brand,A advertorial from the desk of the editor of a publication,,,,,,,,,,,,


In [37]:
import pdfplumber
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def process_text_with_regex(text):
    columns = [
        "Question No", "Question Type", "Objective / Subjective", "Question", "Answer1", "Answer2", "Answer3", "Answer4", "Answer5",
        "CorrectOption", "Difficulty Level", "Audio File Path", "Image Name", "Video File Path", "OptionalSubQuestion",
        "OutOfSubQuestion", "Marks", "ISOR", "IsQuestionWiesRoundOff", "Course Outcomes"
    ]
    data = [
        ["Q1 Main", "Main Question", "Subjective", "Compulsory", "", "", "", "", "", "", "", "", "", "", "", "2", "2", "15", "0", ""]
    ]

    lines = text.split('\n')
    subheader_pattern = re.compile(r'^Q\.\d+(\s|\()')
    question_pattern = re.compile(r'^\d+\.\s')
    sub_question_pattern = re.compile(r'^\([a-cA-C]\)')
    answer_pattern = re.compile(r'^[a-d]\)')
    marks_pattern = re.compile(r'\((\d+)\)')
    additional_info_patterns = {
        "CorrectOption": re.compile(r'CorrectOption:'),
        "Difficulty Level": re.compile(r'Difficulty Level:'),
        "Audio File Path": re.compile(r'Audio File Path:'),
        "Image Name": re.compile(r'Image Name:'),
        "Video File Path": re.compile(r'Video File Path:'),
        "OptionalSubQuestion": re.compile(r'OptionalSubQuestion:'),
        "OutOfSubQuestion": re.compile(r'OutOfSubQuestion:'),
        "Marks": re.compile(r'Marks:'),
        "ISOR": re.compile(r'ISOR:'),
        "IsQuestionWiesRoundOff": re.compile(r'IsQuestionWiesRoundOff:'),
        "Course Outcomes": re.compile(r'Course Outcomes:')
    }

    current_subheader = ""
    question_no = ""
    question = ""
    answers = ["", "", "", "", ""]
    additional_info = ["", "", "", "", "", "", "", "", "", ""]
    solve_placeholder_added = False
    parsing_questions = False
    current_marks = ""

    def add_question_entry():
        nonlocal question_no, question, answers, additional_info, solve_placeholder_added, current_marks
        if question_no:
            data.append([question_no, "Actual Question", "Subjective", question.strip()] + answers + additional_info[:6] + [current_marks] + additional_info[6:])
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            current_marks = ""
        elif current_subheader and not solve_placeholder_added:
            data.append([current_subheader, "Sub Question", "Subjective", "Solve"] + [""] * (len(columns) - 5))
            solve_placeholder_added = True

    for line in lines:
        line = line.strip()
        if not parsing_questions:
            if subheader_pattern.match(line) or question_pattern.match(line):
                parsing_questions = True
            else:
                continue

        if subheader_pattern.match(line):
            add_question_entry()  # Add the previous subheader/questions set if any
            current_subheader = line.split()[0].replace(".", "")
            question_no = ""
            question = ""
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
            solve_placeholder_added = False
            add_question_entry()  # Add the subheader with the "Solve" placeholder
            # Process questions on the same line as the subheader
            remaining_line = " ".join(line.split()[1:])
            sub_question_matches = re.findall(r'\([A-Ca-c]\).*?(?=\([A-Ca-c]\)|$)', remaining_line)
            for match in sub_question_matches:
                sub_question_no = match.split()[0].replace("(", "").replace(")", "")
                sub_question_text = match[len(sub_question_no)+2:].strip()
                marks_match = marks_pattern.search(sub_question_text)
                if marks_match:
                    current_marks = marks_match.group(1)
                data.append([f"{current_subheader}.({sub_question_no})", "Actual Question", "Subjective", sub_question_text, "", "", "", "", "", "", "", "", "", "", "", current_marks, "", "", ""])
                solve_placeholder_added = True
        elif question_pattern.match(line):
            add_question_entry()  # Add the previous question before updating the question number
            question_no = f"{current_subheader}.{line.split()[0].replace('.', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif sub_question_pattern.match(line):
            add_question_entry()  # Add the previous sub-question before updating the sub-question number
            question_no = f"{current_subheader}.{line.split()[0].replace('(', '').replace(')', '')}"
            question = " ".join(line.split()[1:])
            marks_match = marks_pattern.search(question)
            if marks_match:
                current_marks = marks_match.group(1)
            answers = ["", "", "", "", ""]
            additional_info = ["", "", "", "", "", "", "", "", "", ""]
        elif answer_pattern.match(line):
            option = line[0]
            answer = line[2:].strip()
            answers[ord(option) - ord('a')] = answer
        else:
            for key, pattern in additional_info_patterns.items():
                if pattern.match(line):
                    additional_info[columns.index(key) - 5] = line.split(':')[-1].strip()
                    break
            else:
                if question_no:
                    question += " " + line

    add_question_entry()  # Add the last subheader/questions set if any

    # Remove rows where Question No is empty (unnecessary parts)
    df = pd.DataFrame(data, columns=columns)
    df = df[df["Question No"].str.strip().astype(bool)]
    df = df.fillna("")

    # Remove introductory rows
    df = df[~df['Question No'].str.startswith('.')]
    df.reset_index(drop=True, inplace=True)

    # Remove specific phrases from the 'Question' column
    df['Question'] = df['Question'].str.replace(r'\s*\(\d+\)\s*OR\s*', '', regex=True)
    df['Question'] = df['Question'].str.replace(r'\s*\(\d+\)', '', regex=True)
    df['Question'] = df['Question'].str.replace(r'\*\*\*\*\*\*\*\*\*\*\s*1\s*\*\*\*\*\*\*\*\*\*\*', '', regex=True)

    return df

def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

def main():
    pdf_path = "qb - sample.pdf"
    output_excel_path = "Final_output.xlsx"

    text = extract_text_from_pdf(pdf_path)
    df = process_text_with_regex(text)
    save_to_excel(df, output_excel_path)
    return df

df = main()

# Set display options for wider output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(df)


   Question No    Question Type Objective / Subjective                                                                                           Question                                                                            Answer1                                                                Answer2                                                                             Answer3                                                     Answer4 Answer5 CorrectOption Difficulty Level Audio File Path Image Name Video File Path OptionalSubQuestion OutOfSubQuestion Marks ISOR IsQuestionWiesRoundOff Course Outcomes
0      Q1 Main    Main Question             Subjective                                                                                         Compulsory                                                                                                                                                                                                                                  